### Searching for nice test PES

In [ ]:
import numpy as np

import cplot


def f(z):
    return z**4


plt = cplot.plot(
    f,
    (-2.0, +2.0, 400),
    (-2.0, +2.0, 400),
    # abs_scaling=lambda x: x ,  # how to scale the lightness in domain coloring
    # contours_abs=2.0,
    # contours_arg=(-np.pi / 2, 0, np.pi / 2, np.pi),
    # emphasize_abs_contour_1: bool = True,
    # add_colorbars: bool = True,
    # add_axes_labels: bool = True,
    # saturation_adjustment: float = 1.28,
    # min_contour_length = None,
    # linewidth = None,
)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def bump(r2):
    eps = 1e-12
    delta = r2 - 1.0
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.exp(-0.02 / (delta**2 + eps))
        result[np.isclose(delta, 0.0, atol=1e-6)] = 0.0
    return result - 1.0

def V(z, N=6, A=1.0):
    r2 = np.abs(z)**2
    angular = np.real(z**N) / np.abs(z**N) / 2
    return angular
    return bump(r2)
    return bump(r2) * 2.0 + bump(r2) * angular
    return bump(r2) * angular

if __name__ == "__main__":
    N = 6
    A = 2.0
    x = np.linspace(-2, 2, 500)
    y = np.linspace(-2, 2, 500)
    X, Y = np.meshgrid(x, y)
    Z = X + 1j * Y
    ic(X.shape, Y.shape)
    V_val = V(Z, N=N, A=A)
    print(V_val.shape)

    plt.figure(figsize=(6, 5))
    plt.contourf(X, Y, V_val, levels=100, cmap='plasma')
    plt.colorbar(label='V(z)')
    plt.title(f'Localized Potential with N={N}')
    plt.xlabel('Re(z)')
    plt.ylabel('Im(z)')
    plt.gca().set_aspect('equal')
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from icecream import ic

class ArtSurface:
    def __init__(self, num_minima, activation_barrier=2.28, elevation=5.0, ring_width = 0.02):
        self.N = num_minima
        self.activation_barrier = activation_barrier
        self.elevation = elevation
        self.ring_width = ring_width
        self.bump_shape = lambda z: np.abs(z)**2 - 1.0
        
        # # Something more fun than circles:
        # def _f(z):
        #     x, y = np.real(z), np.imag(z)
        #     return (x**2 + y**2)**2 + 4*x*(x**2 + y**2) - 4*y**2
        # def _f(z):
        #     x, y = np.real(z), np.imag(z)
        #     return 2*( x**4-x**2+y**2) + np.exp(-1/(x**2+y**2))
        # self.bump_shape = _f
        # self.bump_shape = lambda z: np.abs(np.real(z) + np.imag(z)*1j*2)**2 - 1.0
        
    
    def _bump(self, z):
        """This is a bump between [-1.0, 0.0] in shape of self.bump_shape"""
        s = self.bump_shape(z)
        eps = 1e-12
        with np.errstate(divide='ignore', invalid='ignore'):
            result = np.exp(-self.ring_width / (s**2 + eps))
            result[np.isclose(s, 0.0, atol=1e-6)] = 0.0
        return 1.0 - result

    def _angular(self, z):
        """Oscillations at origin between [-0.5, 0.5]"""
        return np.real(z**self.N) / np.abs(z**self.N) / 2 + 0.5

    def __call__(self, x, y):
        z = x + 1j * y
        bump = self._bump(z)
        angular = self._angular(z)

        result = angular * bump * self.activation_barrier + (1.0-bump) * self.elevation
        return result
        

In [ ]:
import numpy as np
import plotly.graph_objs as go
from icecream import ic

import matplotlib.pyplot as plt


x = np.linspace(-2, 2, 500)
y = np.linspace(-2, 2, 500)
X, Y = np.meshgrid(x, y)

for i in (1,1.5,2,5,12):
    pes = ArtSurface(i)
    V_val = pes(X, Y)
    print(V_val.shape)
    print(f"min={np.min(V_val)}, max={np.max(V_val)}")

    plt.figure(figsize=(6, 5))
    plt.contourf(X, Y, V_val, levels=100, cmap='BrBG')
    plt.colorbar(label='V(z)')
    plt.title(f'Localized Potential with N={pes.N}')
    plt.xlabel('Re(z)')
    plt.ylabel('Im(z)')
    plt.gca().set_aspect('equal')
    # plt.show()
    plt.savefig(f"model_potentials/{i}.png")

# fig = go.Figure(data=go.Contour(
#     z=V_val,
#     x=x,
#     y=y,
#     colorscale='Plasma',
#     contours_coloring='heatmap',
#     colorbar=dict(title='V(z)'),
#     hovertemplate="x: %{x:.3f}<br>y: %{y:.3f}<br>V: %{z:.4f}<extra></extra>"
# ))

# fig.update_layout(
#     title=f'Localized Potential with N={pes.N}',
#     xaxis_title="Re(z)",
#     yaxis_title="Im(z)",
#     yaxis=dict(scaleanchor="x", scaleratio=1),
#     width=700,
#     height=600,
# )

# fig.show()